In [1]:
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

seed=3
np.random.seed(seed)
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

Using TensorFlow backend.


In [2]:
labels_df = pd.read_csv('product_attributes_updated.csv', encoding='utf-8', 
                    dialect='excel', lineterminator='\n')

In [ ]:
for ColumnName in ['Tier']:
    labels_df[ColumnName].value_counts().plot(kind="bar")
    plt.show()

In [3]:
 def image2np_array( infilename ) :
    img = Image.open( infilename )
    img.load()
    data = np.asarray( img, dtype="int32" )
    return data

x_train_list=[]
y_train_list=[]

one_hot=pd.get_dummies(labels_df['Tier'])

for row in labels_df.itertuples():
    if pd.notnull(row[17]):
        try:
            img = image2np_array('..\\ml_150x150_m\\'+row[1]+'.jpg').reshape(150,150,1)
        except IOError:
            continue
        x_train_list.append(img)
        y_train_list.append(one_hot.loc[row[0],:].values)

In [4]:
x_train=np.array(x_train_list)
y_train=np.array(y_train_list)
len(y_train)==len(x_train)
n_of_training_samples=int(len(x_train)*0.8)
x_validation=x_train[n_of_training_samples:]
y_validation=y_train[n_of_training_samples:]
x_train=x_train[:n_of_training_samples]
y_train=y_train[:n_of_training_samples]

In [5]:
epochs = 20
batch_size = 32

input_shape = (150, 150, 1)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5,seed=seed))
model.add(Dense(4))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
metrics=['accuracy'])

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    zoom_range=0.2,
    horizontal_flip=True)

model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size),
                    validation_data=(x_validation,y_validation),
                    steps_per_epoch=len(x_train) / batch_size, epochs=epochs)

Epoch 1/20
96/95 [==============================] - 6s - loss: 0.5570 - acc: 0.8547 - val_loss: 0.7384 - val_acc: 0.9542
Epoch 2/20
96/95 [==============================] - 4s - loss: 0.4655 - acc: 0.8677 - val_loss: 0.7384 - val_acc: 0.9542
Epoch 3/20
96/95 [==============================] - 4s - loss: 0.3712 - acc: 0.8896 - val_loss: 0.7384 - val_acc: 0.9542
Epoch 4/20
96/95 [==============================] - 4s - loss: 0.3638 - acc: 0.8911 - val_loss: 0.7384 - val_acc: 0.9542
Epoch 5/20
96/95 [==============================] - 4s - loss: 0.3565 - acc: 0.8917 - val_loss: 0.7384 - val_acc: 0.9542
Epoch 6/20
96/95 [==============================] - 4s - loss: 0.3400 - acc: 0.8964 - val_loss: 0.8225 - val_acc: 0.9490
Epoch 7/20
96/95 [==============================] - 4s - loss: 0.3095 - acc: 0.9028 - val_loss: 0.7806 - val_acc: 0.9516
Epoch 8/20
96/95 [==============================] - 4s - loss: 0.3002 - acc: 0.9035 - val_loss: 0.7173 - val_acc: 0.9555
Epoch 9/20
96/95 [==============

In [7]:
model.save('Tier_classifier.h5')